In [263]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [264]:
df=pd.read_csv('train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [265]:
df2=pd.read_csv('test.csv')
df2.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [266]:
y_train=df['Survived']
df.drop('Survived', axis=1, inplace=True)

In [267]:
df.drop(['Name','Ticket','Fare','Cabin'], axis=1, inplace=True)
df2.drop(['Name','Ticket','Fare','Cabin'], axis=1, inplace=True)

In [268]:
dummy=pd.get_dummies(df['Sex'])
df=pd.concat([df,dummy],axis=1)
df.drop(['Sex', 'male'],axis=1,inplace=True)

In [269]:
df.head()

,PassengerId,Pclass,Age,SibSp,Parch,Embarked,female
0,1,3,22.0,1,0,S,0
1,2,1,38.0,1,0,C,1
2,3,3,26.0,0,0,S,1
3,4,1,35.0,1,0,S,1
4,5,3,35.0,0,0,S,0


In [270]:
dummy2=pd.get_dummies(df2['Sex'])
df2=pd.concat([df2,dummy2],axis=1)
df2.drop(['Sex', 'male'],axis=1,inplace=True)

In [271]:
df2.head()

,PassengerId,Pclass,Age,SibSp,Parch,Embarked,female
0,892,3,34.5,0,0,Q,0
1,893,3,47.0,1,0,S,1
2,894,2,62.0,0,0,Q,0
3,895,3,27.0,0,0,S,0
4,896,3,22.0,1,1,S,1


In [272]:
median_ages = np.zeros((2,3))
 
# For each cell in the 2 by 3 matrix
for i in range(0,2):
    for j in range(0,3):
 
    	# Set the value of the cell to be the median of all `Age` values
    	# matching the criterion 'Corresponding gender and Pclass',
    	# leaving out all NaN values
        median_ages[i,j] = df[ (df['female'] == i) & \
                               (df['Pclass'] == j+1)]['Age'].dropna().median()
 
# Create new column AgeFill to put values into. 
# This retains the state of the original data.
df['AgeFill'] = df['Age']
df[ df['Age'].isnull()][['Age', 'AgeFill', 'female', 'Pclass']].head(10)
 
# Put our estimates into NaN rows of new column AgeFill.
# df.loc is a purely label-location based indexer for selection by label.
 
for i in range(0, 2):
    for j in range(0, 3):
        df.loc[ (df.Age.isnull()) & (df.female == i) & (df.Pclass == j+1),'AgeFill'] = median_ages[i,j]

In [273]:
median_ages = np.zeros((2,3))
 
# For each cell in the 2 by 3 matrix
for i in range(0,2):
    for j in range(0,3):
 
    	# Set the value of the cell to be the median of all `Age` values
    	# matching the criterion 'Corresponding gender and Pclass',
    	# leaving out all NaN values
        median_ages[i,j] = df2[ (df2['female'] == i) & \
                               (df2['Pclass'] == j+1)]['Age'].dropna().median()
 
# Create new column AgeFill to put values into. 
# This retains the state of the original data.
df2['AgeFill'] = df2['Age']
df2[ df2['Age'].isnull()][['Age', 'AgeFill', 'female', 'Pclass']].head(10)
 
# Put our estimates into NaN rows of new column AgeFill.
# df.loc is a purely label-location based indexer for selection by label.
 
for i in range(0, 2):
    for j in range(0, 3):
        df2.loc[ (df2.Age.isnull()) & (df2.female == i) & (df2.Pclass == j+1),'AgeFill'] = median_ages[i,j]

In [274]:
df.drop('Age',axis=1,inplace=True)
df2.drop('Age',axis=1,inplace=True)

In [275]:
df.drop('PassengerId',axis=1,inplace=True)
p_id=df2['PassengerId']
df2.drop('PassengerId',axis=1,inplace=True)

In [276]:
dummy=pd.get_dummies(df['Embarked'])
df=pd.concat([df,dummy],axis=1)
df.drop('Embarked',axis=1,inplace=True)

In [277]:
dummy=pd.get_dummies(df2['Embarked'])
df2=pd.concat([df2,dummy],axis=1)
df2.drop('Embarked',axis=1,inplace=True)

In [278]:
import keras
from keras.layers import Dense,Activation
from keras.models import Sequential
from keras.utils import np_utils

In [279]:
model=Sequential()
model.add(Dense(10, input_shape=(8,)))
model.add(Activation('relu'))

model.add(Dense(20))
model.add(Activation('relu'))

model.add(Dense(20))
model.add(Activation('relu'))

model.add(Dense(2))
model.add(Activation('softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_45 (Dense)             (None, 10)                90        
_________________________________________________________________
activation_45 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_46 (Dense)             (None, 20)                220       
_________________________________________________________________
activation_46 (Activation)   (None, 20)                0         
_________________________________________________________________
dense_47 (Dense)             (None, 20)                420       
_________________________________________________________________
activation_47 (Activation)   (None, 20)                0         
_________________________________________________________________
dense_48 (Dense)             (None, 2)                 42        
__________

In [280]:
model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

In [281]:
#One hot encoding of Y
Y=np_utils.to_categorical(y_train)

In [282]:
model.fit(df,Y,epochs=201, verbose=0)

In [283]:
y=model.predict_classes(df2)

In [284]:
y

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [286]:
a = np.array(y, dtype=[('Survived','int64')])
ax=pd.DataFrame(a)
res=pd.concat([p_id,ax],axis=1)
res.to_csv('result.csv')